# SCC5977 - Aprendizado de Máquina para Séries Temporais (2024)

## Grupo
> André Guarnier De Mitri - 11395579 \
> Fabio Cavaleti - 11200550\
> Giovani Decico Lucafó - 10288779

## Problema
Incentia 11k euclidiana vs ddtw

# Teste inicial: KAN

In [1]:
import os
import numpy as np
import torch
from aeon.datasets import load_from_ts_file
from sklearn.preprocessing import LabelEncoder
from kan import KAN

# Load training and testing datasets
if not os.path.exists("./data/ts_files/train.ts") or not os.path.exists("./data/ts_files/test.ts"):
    raise FileNotFoundError("Train or test .ts files not found in the specified directory.")

X_train, y_train = load_from_ts_file("./data/ts_files/train.ts")
X_test, y_test = load_from_ts_file("./data/ts_files/test.ts")

# Encode string labels into integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare data for KAN
train_input = torch.from_numpy(X_train.squeeze(1)).float().to(device)
train_label = torch.from_numpy(y_train_encoded).long().to(device)
test_input = torch.from_numpy(X_test.squeeze(1)).float().to(device)
test_label = torch.from_numpy(y_test_encoded).long().to(device)

# Extract dimensions for KAN
n_instances, n_timepoints = train_input.shape  # 6000, 137
n_classes = len(np.unique(y_train_encoded))  # number of unique classes

# Define the width of the KAN layers (adjusted for your data)
width = [[n_timepoints, 0],
         [50, 0],
         [30, 0],
         [n_classes, 0]]

# Initialize the KAN model
model = KAN(width=width, grid=5, k=3, seed=42, device=device)
model.to(device)

# Prepare dataset dictionary for KAN
dataset = {
    "train_input": train_input,
    "train_label": train_label,
    "test_input": test_input,
    "test_label": test_label,
}

# Accuracy function
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).float())

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).float())


# Train the model with accuracy and F1 score metrics
results = model.fit(
    dataset=dataset,
    steps=100,
    metrics=(train_acc, test_acc),
    loss_fn=torch.nn.CrossEntropyLoss(),
    log=1,
)

# Print the results
train_loss, test_loss = results["train_loss"][-1], results["test_loss"][-1]
train_accuracy, test_accuracy = results["train_acc"][-1], results["test_acc"][-1]

print(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.06e-03 | test_loss: 3.61e+00 | reg: 1.37e+03 | : 100%|█| 100/100 [01:42<00:00,  1.03


saving model version 0.1
Train Loss: 0.0011, Test Loss: 3.6124
Train Accuracy: 1.0000, Test Accuracy: 0.4433


# FCN + KAN

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from models.fcn import FCNClassifier
from utils import TimeSeriesClassifier, TimeSeriesDataset
from kan import KAN
from aeon.datasets import load_from_ts_file

# Load time series data
X_train, y_train = load_from_ts_file("./data/ts_files/train.ts")
X_test, y_test = load_from_ts_file("./data/ts_files/test.ts")

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)

# Infer sequence length and dimensions
sequence_len = X_train.shape[-1]
dimension_num = X_train.shape[1] if len(X_train.shape) > 1 else 1

# Datasets
train_dataset = TimeSeriesDataset(X_train, y_train_encoded)
test_dataset = TimeSeriesDataset(X_test, y_test_encoded)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

# FCNClassifier Model
activation_fn = nn.ReLU()
fcn_model = FCNClassifier(
    dimension_num=dimension_num,
    activation=activation_fn,
    num_classes=num_classes
)

# Wrap FCNClassifier into LightningModule
optimizer = torch.optim.Adadelta(fcn_model.parameters(), lr=1e-3, eps=1e-8)
model_classifier = TimeSeriesClassifier(model=fcn_model, optimizer=optimizer)

# Define checkpoint callback
checkpoint_callback = ModelCheckpoint(
    dirpath="experiments",
    filename="cls_fcn",
    save_top_k=1,
    monitor="train_f1",
    mode="max"
)

# Trainer setup
trainer = Trainer(
    max_epochs=100,
    accelerator="gpu",
    devices=-1,
    callbacks=[checkpoint_callback],
)

# Train the FCNClassifier model
trainer.fit(model_classifier, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\andre\anaconda3\envs\pykan\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\andre\1JUPYTER\SCC5977_MachineLearning_for_TimeSeries\experiments exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | model   | FCNClassifier    | 281 K  | train
1 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
281 K     Trainable params
0         Non-trainable params
281 K     Total params
1.126     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode
c:\Users\andre\anaconda3\envs\pykan\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_d

Epoch 0:   7%|▋         | 7/94 [00:00<00:03, 26.02it/s, v_num=0, train_loss_step=1.130, train_accuracy_step=0.281, train_f1_step=0.205]

c:\Users\andre\anaconda3\envs\pykan\lib\site-packages\torch\nn\modules\conv.py:370: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\Convolution.cpp:1037.)
  return F.conv1d(


Epoch 99: 100%|██████████| 94/94 [00:24<00:00,  3.91it/s, v_num=0, train_loss_step=0.762, train_accuracy_step=0.729, train_f1_step=0.704, train_loss_epoch=0.769, train_accuracy_epoch=0.704, train_f1_epoch=0.702]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 94/94 [00:24<00:00,  3.91it/s, v_num=0, train_loss_step=0.762, train_accuracy_step=0.729, train_f1_step=0.704, train_loss_epoch=0.769, train_accuracy_epoch=0.704, train_f1_epoch=0.702]


In [13]:
from models.fcn import GAP1d

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fcn_model.to(device)
def get_embeddings(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    embeddings_list = []
    labels_list = []

    with torch.no_grad():  # No need to compute gradients
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)  # Ensure both data and labels are on the same device
            
            x = data  # Initialize x with the input data
            for layer in model.layers:
                x = layer(x)
                if isinstance(layer, GAP1d):  # Check if the layer is a GAP1d layer
                    embeddings = x  # Capture the output from GAP1d layer (before flattening)
                    break  # Exit the loop after capturing the embeddings

            embeddings_list.append(embeddings.cpu())  # Store embeddings (move to CPU if necessary)
            labels_list.append(labels.cpu())  # Store labels (move to CPU if necessary)

    embeddings = torch.cat(embeddings_list, dim=0)  # Concatenate embeddings across batches
    labels = torch.cat(labels_list, dim=0)  # Concatenate labels across batches
    
    return embeddings, labels

# Get the embeddings from the FCN model
train_embeddings, train_labels = get_embeddings(fcn_model, train_loader, device)
test_embeddings, test_labels = get_embeddings(fcn_model, test_loader, device)

# Move the embeddings and labels to the correct device (if necessary)
train_input = train_embeddings.to(device)
test_input = test_embeddings.to(device)
train_label = train_labels.to(device)
test_label = test_labels.to(device)

# Define KAN model input dimensions
n_instances, embedding_size = train_input.shape
n_classes = len(np.unique(train_labels))  # Number of classes based on the training labels

width = [[embedding_size, 0],
         [50, 0],
         [30, 0],
         [n_classes, 0]]

# Initialize KAN model
kan_model = KAN(width=width, grid=5, k=3, seed=42, device=device)
kan_model.to(device)

# Prepare dataset for KAN (format for KAN model)
dataset = {
    "train_input": train_input,
    "train_label": train_label,
    "test_input": test_input,
    "test_label": test_label,
}

# Define accuracy functions for KAN
def train_acc():
    return torch.mean((torch.argmax(kan_model(dataset["train_input"]), dim=1) == dataset["train_label"]).float())

def test_acc():
    return torch.mean((torch.argmax(kan_model(dataset["test_input"]), dim=1) == dataset["test_label"]).float())

# Train the KAN model
kan_results = kan_model.fit(
    dataset=dataset,
    steps=50,
    metrics=(train_acc, test_acc),
    loss_fn=torch.nn.CrossEntropyLoss(),
    log=1,
)

# Evaluate the KAN model after training
train_loss, test_loss = kan_results["train_loss"][-1], kan_results["test_loss"][-1]
train_accuracy, test_accuracy = kan_results["train_acc"][-1], kan_results["test_acc"][-1]

# Print results
print(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.08e-01 | test_loss: 1.75e+00 | reg: 9.69e+02 | : 100%|█| 50/50 [01:13<00:00,  1.47s/


saving model version 0.1
Train Loss: 0.1082, Test Loss: 1.7453
Train Accuracy: 0.9982, Test Accuracy: 0.5067
